# Spherepop Playground Notebook

This notebook is a minimal executable toy model of **Spherepop**.

Core ideas demonstrated:
- Spheres are delayed irreversible events.
- `pop()` evaluates exactly once and records history.
- Identity is defined by event history, not by value.
- Nested scopes compose into larger structures.
- Event histories form a geometry that can be visualized.

In [ ]:
import uuid
from dataclasses import dataclass, field
from typing import Callable, Any, List, Tuple, Dict
import matplotlib.pyplot as plt

## Core Spherepop Structures

An **Event** is an irreversible fact with parents (its causal history).  
A **History** is the ledger of all events.  
A **Sphere** is a delayed computation that can be popped once.

In [ ]:
@dataclass
class Event:
    eid: str
    label: str
    value: Any
    parents: Tuple[str, ...]

@dataclass
class History:
    events: Dict[str, Event] = field(default_factory=dict)

    def add(self, event: Event):
        self.events[event.eid] = event

    def lineage(self, eid: str) -> List[str]:
        seen = set()
        order = []
        def dfs(x):
            if x in seen: return
            seen.add(x)
            for p in self.events[x].parents:
                dfs(p)
            order.append(x)
        dfs(eid)
        return order

@dataclass
class Sphere:
    label: str
    thunk: Callable[['History'], Any]
    _eid: str = None

    def pop(self, history: History):
        if self._eid is not None:
            return history.events[self._eid].value, self._eid
        value, parents = self.thunk(history)
        eid = str(uuid.uuid4())
        history.add(Event(eid=eid, label=self.label, value=value, parents=parents))
        self._eid = eid
        return value, eid

## Primitive Constructors

These are the minimal algebra of Spherepop: constants and composition.

In [ ]:
def const(label, x):
    return Sphere(label, lambda h: (x, ()))

def add(label, a: Sphere, b: Sphere):
    def thunk(h):
        va, ea = a.pop(h)
        vb, eb = b.pop(h)
        return va + vb, (ea, eb)
    return Sphere(label, thunk)

def mul(label, a: Sphere, b: Sphere):
    def thunk(h):
        va, ea = a.pop(h)
        vb, eb = b.pop(h)
        return va * vb, (ea, eb)
    return Sphere(label, thunk)

## Example: (1 + 2) × 3

This shows nested irreversible evaluation.

In [ ]:
H = History()
s1 = const('one', 1)
s2 = const('two', 2)
s3 = const('three', 3)
s_add = add('add', s1, s2)
s_expr = mul('mul', s_add, s3)
value, root = s_expr.pop(H)
value

## Identity by History

Same value, different event history ⇒ different identity.

In [ ]:
H2 = History()
a1 = const('one', 1)
a2 = const('two', 2)
a3 = const('three', 3)
expr2 = mul('mul', add('add', a1, a2), a3)
v2, root2 = expr2.pop(H2)
value, v2, root == root2

## Event-History Geometry

We embed the partial order of events as a simple layered graph.

In [ ]:
def depth(h: History, eid: str) -> int:
    e = h.events[eid]
    if not e.parents:
        return 0
    return 1 + max(depth(h, p) for p in e.parents)

nodes = list(H.events.keys())
depths = {eid: depth(H, eid) for eid in nodes}
layers = {}
for eid, d in depths.items():
    layers.setdefault(d, []).append(eid)

pos = {}
for d, eids in layers.items():
    for i, eid in enumerate(eids):
        pos[eid] = (i, -d)

plt.figure()
for eid, e in H.events.items():
    x, y = pos[eid]
    plt.scatter([x], [y])
    plt.text(x+0.02, y+0.02, e.label)
    for p in e.parents:
        xp, yp = pos[p]
        plt.plot([xp, x], [yp, y])
plt.title('Spherepop Event-History Geometry')
plt.axis('off')
plt.show()